In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import sqlalchemy
import os
import json
import uuid

In [2]:
conn = psycopg2.connect(
    user="dcat", 
    password="dcat", 
    host="127.0.0.1", 
    port="54320", 
    database="dcat")

In [3]:
cur = conn.cursor()

In [4]:
sql = "SELECT * FROM tbl_node WHERE prop->>'id' = 'GDPR.PROCESSING_ACTIVITY.UFØRETRYGD'"
df = sqlio.read_sql_query(sql, conn)
df

,id,prop,created
0,299,"{'id': 'GDPR.PROCESSING_ACTIVITY.UFØRETRYGD', ...",2020-01-08 08:25:35.474422


In [11]:
edges = [
  {
    "n1": 299,
    "n2": 300,
    "prop": ["part of", "belongs to"]
  }
]

statement = "INSERT INTO tbl_edge (n1, n2, prop) VALUES "

for edge_item in edges:
        n1 = edge_item.get("n1")
        n2 = edge_item.get("n2")
        prop = json.dumps(edge_item.get("prop"))
        if n1 is None:
            abort(409, f"The edge must have a source node n1 of type string")
        if n2 is None:
            abort(409, f"The edge must have a target node n2 of type string")
        if prop is None:
            abort(409, f"The edge must have a prop of type string")
        else:
            statement = statement + f"({n1}, {n2}, '{prop}'), "

# Deleting the space and ',' at the end of the statement
statement = statement[:-2]
# On receiving a pair of nodes that already exist it will add the prop to the prop array
statement = statement + " ON CONFLICT (n1, n2) DO UPDATE SET n1 = excluded.n1, n2 = excluded.n2, prop=excluded.prop"
print(statement)

INSERT INTO tbl_edge (n1, n2, prop) VALUES (299, 300, '["part of", "belongs to"]') ON CONFLICT (n1, n2) DO UPDATE SET n1 = excluded.n1, n2 = excluded.n2, prop=excluded.prop


In [14]:
res = cur.execute(statement)
print(res)
conn.commit()

None


In [15]:
sql = "SELECT * FROM tbl_edge WHERE n1 = 299"
df = sqlio.read_sql_query(sql, conn)
df

,n1,n2,prop,created
0,299,300,"[part of, belongs to]",2020-01-08 08:31:01.918221


In [16]:
conn.commit()

In [17]:
inp = {
  "prop": {"id":"test0","type": "test6"}
}

jsonb = json.dumps(inp['prop'])
sql = f"INSERT INTO tbl_node (id, prop) VALUES (270,'{jsonb}') ON CONFLICT (id) DO UPDATE SET prop  = '{jsonb}';"
cur.execute(sql, (jsonb,))
conn.commit()